In [1]:
import cv2
import numpy as np
from PIL import ImageGrab
import time

# Load Yolo
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))


In [2]:
# # Loading image
# img = cv2.imread("kiosk.jpg")
# img = cv2.resize(img, None, fx=1, fy=1)
# height, width, channels = img.shape

In [3]:
def screen_record(): 
#     global out
    last_time = time.time()    
    while(True):
        printscreen =  np.array(ImageGrab.grab(bbox=(20,250,1070, 900)))

#         print('loop took {} seconds'.format(time.time()-last_time))
        new_screen = process_img(printscreen, last_time)
        last_time = time.time()
        
        cv2.imshow('window',cv2.cvtColor(printscreen, cv2.COLOR_BGR2RGB))
        if cv2.waitKey(1) & 0xFF == ord('q'):
#         if cv2.waitKey(0):

#             out.release()
            cv2.destroyAllWindows()
            break
            
            

In [4]:
def vid_stream():
    cap = cv2.VideoCapture("walking.mp4")
    last_time = time.time()    
    frame_id = 0
    
    while True:
        _, printscreen = cap.read()
        frame_id += 1
        last_time = time.time()        

#         print('loop took {} seconds'.format(time.time()-last_time))
        new_screen = process_img(printscreen, last_time)
        last_time = time.time()
        
#         cv2.imshow('window',cv2.cvtColor(printscreen, cv2.COLOR_BGR2RGB))
        cv2.imshow('window',printscreen)
        if cv2.waitKey(1) & 0xFF == ord('q'):
#         if cv2.waitKey(0):

#             out.release()
            cv2.destroyAllWindows()
            break

In [5]:
# cv2.imshow('random string', img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [19]:
def process_img(img, last_time):
#     width=1050
#     height=650
    height, width, channels = img.shape
    
    # blob size determines speed & accuracy
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), swapRB=True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    class_ids = []
    confidences = []
    boxes = []
#     peopleCount=0
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)

            confidence = scores[class_id]
            if confidence > 0.3:

#                 if classes[class_id]=="person": 
#                     peopleCount += 1
# #                     print(peopleCount)

                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.3, 0.27)
    # print(indexes)
    font = cv2.FONT_HERSHEY_PLAIN
    peopleCount=0
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            if label == "person": 
                peopleCount += 1
#                 print(peopleCount)
            
            
#             print(label)
            color = colors[i]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label + " " + str(round(confidence,2)), (x, y + 30), font, 1, color, 2)

    timeTaken = 'FPS: {}'.format(round(1/(time.time()-last_time),2))
            
            
    cv2.putText(img, "pax count: " + str(peopleCount),(20,height-50), font, 2, (0,0,255), 2)
    cv2.putText(img, timeTaken,(20,height-10), font, 2, (0,0,255), 2)

    return(img)

In [20]:
# screen_record()
vid_stream()